In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import numpy as np
import hashlib

# Ladda in data
athletes = pd.read_csv("athlete_events.csv")
noc = pd.read_csv("noc_regions.csv")

In [5]:
# Slå ihop dataset för att få med regionnamn
data_raw = pd.merge(athletes, noc, on="NOC", how="left")

**UPPGIFT 1** - 
Annonymiserar alla namn med hashfunktionen SHA-256:

In [6]:
# Tar en kopia av originaldatasetet
data = data_raw.copy()

# Funktion för SHA-256-hash av namn (med salt)
def hash_name(name: str, salt: str = "") -> str:
    if pd.isna(name):
        return np.nan
    text = (salt + name.strip().lower()).encode("utf-8")
    return hashlib.sha256(text).hexdigest()

# Salt för hashning (byt gärna till något eget/projektspecifikt)
SALT = "projekt_os_salt"

# Skapa hash-kolumn för ALLA deltagare
data["Name_hash"] = data["Name"].map(lambda x: hash_name(x, SALT))

# Ta bort originalnamnet för att anonymisera
data = data.drop(columns=["Name"])

# Spara som ny fil (hela datasetet anonymiserat)
data.to_csv("data_anonymized.csv", index=False)

# Dubbelkolla resultatet
print(data[["NOC", "Name_hash"]].head())
print("Antal unika hashade individer:", data["Name_hash"].nunique())

   NOC                                          Name_hash
0  CHN  e71264f789929eef276224b1e25d5e185f119ca2babeb2...
1  CHN  350bf0f45aa9097774cb74c13c00a4aeb5bf1bf7fbe965...
2  DEN  6f9c7d628645d2e9cb62f48f95c92d4c1c4fd9d87363c3...
3  DEN  0d4fe3228bf1e6c7fbb70439640aa9a0095f8bdccaf2fb...
4  NED  6fceadb641dfc1e3230bf9d5fb7f6753142deac33ccb43...
Antal unika hashade individer: 134730


1. Delal: Översikt deltagande(antingen utifrån ID eller Event) - varför fler eller färre vissa år (krig/politik)? (linjegraf). Ungern vs alla länder.  - Övergång till prestation

2. Patrik: Sporter de är starkast i, antal medaljer per OS (vinter vs sommar)- övergång till bästa året 

3. Adrian: 1952 - bästa året, varför? Deltagande-  Vilken sport bäst (två minst)? - Art competition inte kvar som Sport(dubbletter). Övergång till kön

In [ ]:
#Total medals by sport 1952

hun = data[data["NOC"] == "HUN"]
hun52 = hun[hun["Year"] == 1952]


Unique_Medal_Event52 = hun52.drop_duplicates(subset=["Sport", "Event", "Medal"])
Unique_Medals_Sport52 = Unique_Medal_Event52.groupby(["Sport", "Medal"]).size().unstack(fill_value=0)
Unique_Medals_Sport52["Total"] = Unique_Medals_Sport52[["Gold", "Silver", "Bronze"]].sum(axis=1)
Unique_Medals_Sport52.sort_values(by= "Total", ascending=False, inplace=True)

hun_top10_Sports_52 = Unique_Medals_Sport52[:10].reset_index()

fig = px.bar(hun_top10_Sports_52, x= "Total", y= "Sport", title="Unique medals per sport 1952", orientation= "h", color= "Total")
fig.update_layout(yaxis=dict(autorange="reversed"))
fig.show()

4. Fanny: kön kvinnor vs män 1952 - gymnastik, annan sport, annat land eller annat år, har fördelningen förändrats genom tiderna?

In [ ]:
import plotly.io as pio # bara tillfällig för jupyter (behövs inte sedan)
pio.renderers.default = "browser" # bara tillfällig för jupyter (behövs inte sedan)


# instead of plotly.express due to eaiser customization options
# and better overview of the code (for this specific plot)
import plotly.graph_objects as go    

df = data

hun = df[df["NOC"] == "HUN"]
hun = hun[hun["Year"].between(1920, 1968)]

# groups the data by Year and Sex, and counts medalists and non-medalists. 
summary = (
    hun
    .groupby(["Year", "Sex"])
    .agg(
        Medalists=("Medal", lambda s: s.notna().sum()), # new column for medalists
        Non_medalists=("Medal", lambda s: s.isna().sum()) # new column for non-medalists
    )
    .reset_index() # resets index after grouping to get Year and Sex as columns   
)

# to get separate bars for F and M (year + sex)   
summary["Year_sex"] = summary["Year"].astype(str) + " " + summary["Sex"]

fig = go.Figure()

# same color for both M and F medalists
fig.add_bar(
    x=summary["Year_sex"],
    y=summary["Medalists"],
    name="Medalists",
    marker_color="gray"
)

# F without medals = red
fig.add_bar(
    x=summary.loc[summary["Sex"]=="F", "Year_sex"],
    y=summary.loc[summary["Sex"]=="F", "Non_medalists"],
    name="F - no medals",
    marker_color="red"
)

# M without medals = green
fig.add_bar(
    x=summary.loc[summary["Sex"]=="M", "Year_sex"],
    y=summary.loc[summary["Sex"]=="M", "Non_medalists"],
    name="M - no medals",
    marker_color="green"
)

fig.update_layout(
    barmode="stack",
    title="Hungarian Olympic participants by year and sex (+ number of medals per group)",
    xaxis_title="Year and sex",
    yaxis_title="Number of perticipants" 
)

fig.show()


In [8]:

# Könsfördelning alla år i Gymnastik, jämförelse mellan alla länder, Ungern och Sverige

pio.renderers.default = "browser"  # Tillfälligt, får inte upp i nb.

gymnastics_all = data[data["Sport"] == "Gymnastics"] 
gymnastics_hun = data[data["Sport"] == "Gymnastics"][data["NOC"] == "HUN"]  
gymnastics_swe = data[data["Sport"] == "Gymnastics"][data["NOC"] == "SWE"]   

# Lägger till en kolumn som talar om grupp
all_gymnastics_group = gymnastics_all.assign(Group="Alla länder")
hun_gymnastics_group = gymnastics_hun.assign(Group="Ungern")
swe_gymnastics_group = gymnastics_swe.assign(Group="Sverige")

# Slå ihop
gymnastics_combined = pd.concat([all_gymnastics_group, hun_gymnastics_group, swe_gymnastics_group])

# Gör cirkeldiagram
fig = px.pie(
    gymnastics_combined,
    names="Sex",
    facet_col="Group",        # en subplot per grupp
    category_orders={"Group": ["Alla länder", "Ungern", "Sverige"]},  # ordning
    title="Könsfördelning i Gymnastik alla år"
)

fig.show()


C:\Users\fanny\AppData\Local\Temp\ipykernel_20400\616710349.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\fanny\AppData\Local\Temp\ipykernel_20400\616710349.py:7: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [9]:
# Könsfördelning i Gymnastik OS 1952, jämförelse mellan alla länder, Ungern och Sverige  

pio.renderers.default = "browser"  # Tillfälligt, får inte upp i nb.

gymnastics_all_1952 = data[data["Sport"] == "Gymnastics"][data["Year"] == 1952]
gymnastics_hun_1952 = data[data["Sport"] == "Gymnastics"][data["NOC"] == "HUN"][data["Year"] == 1952]
gymnastics_swe_1952 = data[data["Sport"] == "Gymnastics"][data["NOC"] == "SWE"][data["Year"] == 1952]     

# Lägger till en kolumn som talar om grupp
all_gymnastics_group = gymnastics_all_1952.assign(Group="Alla länder")
hun_gymnastics_group = gymnastics_hun_1952.assign(Group="Ungern")
swe_gymnastics_group = gymnastics_swe_1952.assign(Group="Sverige")

# Slå ihop
gymnastics_combined = pd.concat([all_gymnastics_group, hun_gymnastics_group, swe_gymnastics_group])

# Gör cirkeldiagram
fig = px.pie(
    gymnastics_combined,
    names="Sex",
    facet_col="Group",        # en subplot per grupp
    category_orders={"Group": ["Alla länder", "Ungern", "Sverige"]},  # ordning
    title="Könsfördelning i Gymnastik OS 1952 (Helsingfors)"
)

fig.show()

C:\Users\fanny\AppData\Local\Temp\ipykernel_20400\2926502224.py:5: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\fanny\AppData\Local\Temp\ipykernel_20400\2926502224.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\fanny\AppData\Local\Temp\ipykernel_20400\2926502224.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\fanny\AppData\Local\Temp\ipykernel_20400\2926502224.py:7: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\fanny\AppData\Local\Temp\ipykernel_20400\2926502224.py:7: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



**Checklista:** 

- Vem tar vad? 
- frågeställningar - Vilken sport har Ungern varit starkast i? Vilka historiska händelser har påverkat? Hur påverkar Art competition helhetsresultatet? Varför så många deltagenade/medaljer 1952? Hur ser skillnaden ut i kön genom åren jämfört med andra länder? 
- Dash-applikationen
- "load data-filen"
- Layout
- Annonymiseringen 
- Render 
- Vilka grafer? 
- Rensa i repot - vad kan tas bort?  

**Gemensamma filer:**  (kom ihåg att källhänvisa till kod som kopierats)  
- Jupyter notebook, med allt vi gjort + beskrivningar.  
- Dash applikation (main, data/funktioner, layout)
- Datafiler (athletes, NOC och annonymiserad data)

**I presentationen** (Detta ska förklaras)  
- vilka frågeställningar ni haft.
- hur ni gått tillväga för att besvara dessa frågeställningar.
- varför ni valt just de olika diagrammen/graferna
- varför ni designat dashboardapplikationen som ni gjort. Förslag är att kort beskriva er dataanalys, följt av dashboardapplikationen

ALLTID PULL FÖRST
Kollar igenom uppgift 0 på egen hand. 

Tisdag? 
Onsdag efter föreläsningen. 

Test /PH

Inför nästa (uppgift 2)
  
Patrik - Tyngdlyftning  
Adrian - Gymnastik  
Fanny - Tennis (fencing)  
Delal - Figure skating   

Tema: Historiska händelser och dess påverkan på resultat och deltagande. 
Män/kvinnor (deltagande vid krigsår och över tid, jämfört med andra länder), Krig (ålder och deltagande),  

Variabler  
- Antal medaljer (guld, silver och brons) Jämfört med varandra och andra länder (stapeldiagram, barmode=relative) (boxplot : hur många medaljer per kön)
- årtal (sticker några årtal ut?) - px.bar (barplot)
- ålder (Jämföra sporter emellan, histogram subplot) (Åldersskillnad beroende på krig? Medelåldern/median = linjediagram)  
- kön (inom respektive sport, jämfört med varandra(våra sporter), jämfört med andra länder inom samma sport) Cirkeldiagram (subplots för att visa skillnaden mellan sporter/länder)

Stapeldiagram: Kvinnor män per sport = en stapel = kvinna eller man /sport  

Förslag, är ni med på detta? och i så fall, säg vilken kategori ni vill ha: 

Kategorier: 
Kön - Fanny
Ålder - 
Medaljer - 
Deltagande - 

Utgå från historiska händelser/utveckling genom åren för Ungern och hur det påverkat deras medverkan i OS utifrån de olika kategorierna. Två sporter: fencing och weightlifting? 


Ungern i sin helhet - Vinter vs sommar OS. Jämföra med annat land (strkt vinter OS-land). 

Dashboeard url:

https://project-olympics-hungary.onrender.com

In [ ]:
print("Test")

Test


In [ ]:
import pandas as pd

# Ladda in data
athletes = pd.read_csv("athlete_events.csv")
noc = pd.read_csv("noc_regions.csv")

# Slå ihop dataset för att få regionnamn
data = pd.merge(athletes, noc, on="NOC", how="left")

# Filtrera på Ungern
hungary = data[data["region"] == "Hungary"]
sweden = data[data["region"] == "Sweden"]
usa = data[data["region"] == "USA"]


**årtal som sticker ut**  
  
**1916** - Ungern deltog inte i OS 1916, eftersom spelen aldrig hölls.  
De var planerade att äga rum i Berlin, men ställdes in på grund av första världskriget.  
  
**1920** - Ungen förbjöds att vara med pga sin roll och inblandning i första världskriget.  
  
(**1980** - USA ledde en bojkott av de Olympiska spelen i Moskva som en protest mot Sovjets invasion av Afghanistan. Ytterligare 60 länder deltog i bojkotten, bland annat Kina.)     
  
**1984** - Ungern bojkottade OS 1984 i Los Angeles, tillsammans med andra öststatländer, som svar på den USA-ledda bojkotten av OS i Moskva 1980. Bojkotten var en politisk handling som genomfördes på Sovjetunionens begäran. 
Orsak: Bojkotten var en vedergällning för den bojkott som USA och andra länder genomförde mot OS i Moskva 1980. (kalla kriget?)
Beslut: Ungern deltog inte eftersom de, likt andra socialistiska länder, följde Sovjetunionens beslut.  
    
***Kod där man ser tecken på detta utifrån antal deltagare:*** 

In [ ]:
year_16_hun = hungary[hungary["Year"] == 1916]
year_16_all = athletes[athletes["Year"] == 1916]

year_20_hun = hungary[hungary["Year"] == 1920]
year_20_swe = sweden[sweden["Year"] == 1920]

year_80_usa = usa[usa["Year"] == 1980]
year_84_usa = usa[usa["Year"] == 1984]

year_84_hun = hungary[hungary["Year"] == 1984]
year_84_swe = sweden[sweden["Year"] == 1984]
year_88_hun = hungary[hungary["Year"] == 1988]

print(f"Antal deltagare Ungern 1916: {len(year_16_hun)}")
print(f"Antal deltagare totalt 1916: {len(year_16_all)}")

print(f"Antal deltagare Ungern 1920: {len(year_20_hun)}") 
print(f"Antal deltagare Sverige 1920: {len(year_20_swe)}")

print(f"Antal deltagare USA 1980: {len(year_80_usa)}")
print(f"Antal deltagare USA 1984: {len(year_84_usa)}")

print(f"Antal deltagare Ungern 1984: {len(year_84_hun)}")
print(f"Antal deltagare Sverige 1984: {len(year_84_swe)}")

print(f"Antal deltagare Ungern 1988: {len(year_88_hun)}")



Antal deltagare Ungern 1916: 0
Antal deltagare totalt 1916: 0
Antal deltagare Ungern 1920: 0
Antal deltagare Sverige 1920: 404
Antal deltagare USA 1980: 155
Antal deltagare USA 1984: 849
Antal deltagare Ungern 1984: 16
Antal deltagare Sverige 1984: 352
Antal deltagare Ungern 1988: 322
